# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [15]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [16]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f'Only {count} establishments have a hygiene score of 20')

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

Only 0 establishments have a hygiene score of 20
None


In [7]:
# Convert the result to a Pandas DataFrame
hygene_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(f'There are {len(hygene_df)} rows')

# Display the first 10 rows of the DataFrame
print(f'There are {len(hygene_df)} rows')

There are 0 rows
There are 0 rows


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'}, 
          'RatingValue': {'$gte': 4} }

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f' {count} establishments in London have a Rating Value of 4 or more')

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

 0 establishments in London have a Rating Value of 4 or more
None


In [9]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(f'There are {len(london_df)} rows')

# Display the first 10 rows of the DataFrame
london_df.head(10)

There are 0 rows


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
        'RatingValue': 5, 
        'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
        'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search}
    }

sort =  [('scores.Hygiene', 1)]

# Print the results
top = []
for result in establishments.find(query).sort(sort).limit(5):
    top.append(result)

pprint(top)

[]


In [11]:
# Convert result to Pandas DataFrame
top_df = pd.DataFrame(top)
top_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match':{'scores.Hygiene': 0}}
group_query = {'$group':{'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}
sort_values = {'$sort':{'count': -1}}

pipeline = [match_query, group_query, sort_values]

bad_hygene = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f'There are {len(bad_hygene)} results')

# Print the first 10 results
pprint(bad_hygene[0:10])

There are 0 results
[]


In [13]:
# Convert the result to a Pandas DataFrame
hygene_df = pd.DataFrame(bad_hygene)

# Display the number of rows in the DataFrame
print(f'There are {len(hygene_df)} rows')

# Display the first 10 rows of the DataFrame
hygene_df.head(10)

There are 0 rows


""
